In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [25]:
file_paths = ['01_SSP1_데이터.csv', '02_SSP3_데이터.csv', '03_SSP5_데이터.csv']

In [26]:
# 파일 경로와 해당 시나리오 이름을 매핑하는 딕셔너리를 생성합니다.
# 이 구조는 새로운 시나리오(예: SSP2, SSP4)가 추가될 때 코드 수정 없이
# 딕셔너리에 항목만 추가하면 되므로 확장성이 매우 뛰어납니다.
scenario_mapping = {
    '01_SSP1_데이터.csv': 'SSP1',
    '02_SSP3_데이터.csv': 'SSP3',
    '03_SSP5_데이터.csv': 'SSP5'
}

# 개별적으로 처리된 데이터프레임을 저장할 빈 리스트를 초기화합니다.
list_of_dataframes = []

# 딕셔너리를 순회하며 각 파일을 처리합니다.
for file_path, scenario_name in scenario_mapping.items():
    # CSV 파일을 데이터프레임으로 읽어옵니다.
    df = pd.read_csv(file_path)
    
    # 'Scenario'라는 새로운 열을 추가하고, 해당 시나리오 이름으로 값을 채웁니다.
    # 이 작업은 데이터가 다른 출처와 섞이기 전에 수행되어 출처의 무결성을 보장합니다.
    df['Scenario'] = scenario_name
    
    # 처리된 데이터프레임을 리스트에 추가합니다.
    list_of_dataframes.append(df)

print(f"총 {len(list_of_dataframes)}개의 데이터프레임이 성공적으로 처리 및 태깅되었습니다.")

총 3개의 데이터프레임이 성공적으로 처리 및 태깅되었습니다.


In [27]:
# pd.concat 함수를 사용하여 리스트에 있는 모든 데이터프레임을 수직으로 결합합니다.
# 이 함수는 스키마가 동일한 데이터프레임들을 효율적으로 쌓아줍니다.
merged_df = pd.concat(list_of_dataframes, ignore_index=True)

# ignore_index=True 인자는 매우 중요합니다.
# 이 옵션은 기존의 개별 인덱스를 무시하고, 0부터 시작하는 연속적인 새 인덱스를
# 최종 데이터프레임에 부여합니다. 이는 데이터 슬라이싱, 필터링 등
# 후속 분석 작업의 일관성과 편의성을 위해 필수적입니다.

print("데이터프레임 병합 완료.")
print("최종 데이터셋의 형태(행, 열):", merged_df.shape)

데이터프레임 병합 완료.
최종 데이터셋의 형태(행, 열): (405, 26)


In [28]:
# 원본 파일들의 총 행 수를 계산하여 예상되는 최종 행 수를 구합니다.
expected_rows = 0
for df in list_of_dataframes:
    expected_rows += len(df)

# 최종 병합된 데이터프레임의 실제 차원을 확인합니다.
actual_rows, actual_cols = merged_df.shape

# 예상되는 열의 수는 원본 25개 + 새로 추가된 'Scenario' 열 1개 = 26개입니다.
expected_cols = 26

# 검증 결과를 출력합니다.
print(f"예상 행 수: {expected_rows}, 실제 행 수: {actual_rows}")
print(f"예상 열 수: {expected_cols}, 실제 열 수: {actual_cols}")

# 자동 검증 로직
if actual_rows == expected_rows and actual_cols == expected_cols:
    print("차원 검증 통과: 데이터의 손실이나 불필요한 추가 없이 병합되었습니다.")
else:
    print("차원 검증 실패: 데이터 처리 과정에 오류가 있을 수 있습니다.")

예상 행 수: 405, 실제 행 수: 405
예상 열 수: 26, 실제 열 수: 26
차원 검증 통과: 데이터의 손실이나 불필요한 추가 없이 병합되었습니다.


In [35]:
import pandas as pd
import os

# --- 1. 파일 준비 ---
file_paths = ['01_SSP1_데이터.csv', '02_SSP3_데이터.csv', '03_SSP5_데이터.csv']
list_of_dataframes = []

print("--- 데이터 불러오기 및 병합 시작 ---")

# --- 2. 반복문으로 파일 불러오기 및 'scenario' 컬럼 추가 ---
for path in file_paths:
    # CSV 파일을 데이터프레임으로 읽어옵니다.
    df = pd.read_csv(path)
    
    # 파일명에서 시나리오 정보(예: SSP1)를 추출합니다.
    scenario_name = os.path.basename(path).split('_')[1]
    
    # 'scenario'라는 새로운 컬럼을 만들고 추출한 시나리오 이름을 값으로 할당합니다.
    # ★★★ 여기가 가장 중요! 'scenario' 컬럼이 여기서 생성됩니다. ★★★
    df['scenario'] = scenario_name
    
    # 처리된 데이터프레임을 리스트에 추가합니다.
    list_of_dataframes.append(df)

# --- 3. 데이터프레임 하나로 합치기 ---
merged_df = pd.concat(list_of_dataframes, ignore_index=True)

print("데이터프레임 병합 완료.")
print("최종 데이터셋의 형태(행, 열):", merged_df.shape)


# --- 4. 디버깅: 병합 후 컬럼명과 데이터 확인 (오류 원인 찾기) ---
print("\n--- [디버깅] 병합 후 실제 컬럼명 확인 ---")
print(merged_df.columns) # 이 출력을 통해 'scenario' 컬럼이 잘 만들어졌는지 확인!

print("\n--- [디버깅] 병합 후 데이터 상위 5개 확인 ---")
print(merged_df.head()) # 'scenario' 컬럼과 값이 잘 들어갔는지 눈으로 확인!


# --- 5. 최종 검증 ---
print("\n--- 데이터 병합 검증 시작 ---")

# 'scenario' 열의 각 고유값과 그에 해당하는 행의 수를 계산합니다.
# 이 코드가 제대로 실행되려면 바로 위에서 'scenario' 컬럼이 정상적으로 보여야 합니다.
scenario_counts = merged_df['scenario'].value_counts()

print("\n'scenario' 열의 값 분포:")
print(scenario_counts)

# 원본 데이터와 비교하여 검증합니다.
for original_df, path in zip(list_of_dataframes, file_paths):
    scenario_name = os.path.basename(path).split('_')[1]
    original_row_count = len(original_df)
    merged_row_count = scenario_counts[scenario_name]
    
    if original_row_count == merged_row_count:
        print(f"✅ [{scenario_name}] 검증 통과: 원본({original_row_count} 행) == 병합 후({merged_row_count} 행)")
    else:
        print(f"❌ [{scenario_name}] 검증 실패: 원본({original_row_count} 행) != 병합 후({merged_row_count} 행)")

--- 데이터 불러오기 및 병합 시작 ---
데이터프레임 병합 완료.
최종 데이터셋의 형태(행, 열): (405, 26)

--- [디버깅] 병합 후 실제 컬럼명 확인 ---
Index(['month', 'CO2ppm', 'Temp', 'Humid', 'VPD', 'Chl_a', 'Chl_b', 'TChl',
       'Car', 'Chl_a_b', 'TCh-Car', 'ABS-RC', 'Dio-RC', 'Tro-RC', 'Eto-RC',
       'PI_abs', 'DF_abs', 'SFI_abs', 'Fv-Fm', 'Leaf_ExtractionYield',
       'Root_ExtractionYield', 'Leaf_TPC', 'Root_TPC', 'Leaf_TFC', 'Root_TFC',
       'scenario'],
      dtype='object')

--- [디버깅] 병합 후 데이터 상위 5개 확인 ---
   month      CO2ppm       Temp      Humid       VPD  Chl_a  Chl_b   TChl  \
0      5  381.681033  16.918639  83.130786  1.532512   8.79   2.22  11.00   
1      5  374.463441  16.922124  83.096722  1.532868   8.99   2.56  11.55   
2      5  371.850683  16.930256  82.488003  1.534584   9.66   2.44  12.10   
3      5  400.475202  16.921511  82.081632  1.534512   9.33   2.45  11.79   
4      5  381.360788  16.921323  83.888666  1.531475  10.53   2.58  13.11   

    Car  Chl_a_b  ...  DF_abs  SFI_abs  Fv-Fm  Leaf_Extraction

In [36]:
merged_df.to_csv('df_merged.csv', index=False, encoding='utf-8-sig')